In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dlp-may-2025-nppe-3/sample_submission.csv
/kaggle/input/dlp-may-2025-nppe-3/submission.py
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00159.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00056.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00017.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00124.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00140.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00068.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00019.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00266.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00236.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00148.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00152.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00226.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00008.png
/kaggle/input/dlp-may-2025-nppe-3/archive/val/gt/gt_00216.png
/kaggle/input/dlp-may-2025-n

In [2]:
# ============================================================
# NPPE-3: Low-Light Denoising + 4× Super-Resolution Baseline
# Model: Hugging Face Real-ESRGAN x4 (pretrained)
# Denoiser: OpenCV fastNlMeansDenoisingColored
# Submission: via provided submission.py
# ============================================================

In [3]:
# 0) CONFIG (EDIT YOUR ROLL NO)
# -----------------------------
ROLL_NO = "21F1005989"  # TODO: change to your roll number (used in final submission.csv name)
TEST_DIR = "/kaggle/input/dlp-may-2025-nppe-3/archive/test"  # TODO: replace with correct Kaggle test folder
OUTPUT_DIR = f"/kaggle/working/{ROLL_NO}_images"
FINAL_SUB_CSV = f"/kaggle/working/submission.csv"

In [4]:
# 1) INSTALL DEPENDENCIES
# -----------------------------
!pip -q install realesrgan basicsr facexlib gfpgan opencv-python pillow tqdm huggingface_hub stablepy torchvision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 200.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 16.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.5/262.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.9/261.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 58.8 M

In [5]:
# 2) IMPORTS
# -----------------------------
import os, cv2, numpy as np
from tqdm import tqdm
from PIL import Image
import torch
from huggingface_hub import hf_hub_download
# from realesrgan import RealESRGAN

In [6]:
# 3) DEVICE
# -----------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Device: cuda


In [7]:
# 4) PREP OUTPUT DIR
# -----------------------------
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [8]:
# 5) Load Real-ESRGAN Anime Fine-Tuned via stablepy
from huggingface_hub import hf_hub_download
from stablepy import load_upscaler_model
import torch

print("Downloading Anime-fine-tuned Real-ESRGAN model...")
anime_weights = hf_hub_download(
    repo_id="danhtran2mind/Real-ESRGAN-Anime-finetuning",
    filename="Real-ESRGAN-Anime-finetuning.pth"
)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Initializing stablepy upscaler with anime-finetuned weights...")
upscaler = load_upscaler_model(
    model=anime_weights,
    tile=0,
    tile_overlap=8,
    device=device,
    half=(device == "cuda")
)
print("✅ Anime Real-ESRGAN loaded successfully")


2025-08-22 18:08:42.324971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755886122.653251      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755886122.751093      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Real-ESRGAN-Anime-finetuning.pth:   0%|          | 0.00/134M [00:00<?, ?B/s]

Initializing stablepy upscaler with anime-finetuned weights...
✅ Anime Real-ESRGAN loaded successfully


In [9]:
# 6) HELPER FUNCTIONS
# -----------------------------
def denoise_bgr(bgr_img):
    return cv2.fastNlMeansDenoisingColored(bgr_img, None, 7, 7, 7, 21)

def pil_to_bgr(im: Image.Image):
    return cv2.cvtColor(np.array(im.convert("RGB")), cv2.COLOR_RGB2BGR)

def bgr_to_pil(bgr):
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return Image.fromarray(rgb)

In [10]:
# 7) INFERENCE PIPELINE
# -----------------------------
test_images = sorted(os.listdir(TEST_DIR))
print(f"Found {len(test_images)} test images")

for fname in tqdm(test_images):
    if not fname.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    im = Image.open(os.path.join(TEST_DIR, fname)).convert("RGB")

    # 1. denoise
    bgr = pil_to_bgr(im)
    bgr_denoised = denoise_bgr(bgr)
    denoised_pil = bgr_to_pil(bgr_denoised)

    # 2. super-resolution (4×)
    with torch.inference_mode():
        pil_im = denoised_pil.convert("RGB")  # ensure correct mode
        sr_img = upscaler.upscale(pil_im, 4)  # 4× super-resolution
        # sr_img = sr_model.predict(denoised_pil)

    # save
    out_path = os.path.join(OUTPUT_DIR, fname)
    sr_img.save(out_path)

print("✅ All test images processed and saved at:", OUTPUT_DIR)

Found 60 test images


100%|██████████| 60/60 [00:57<00:00,  1.04it/s]

✅ All test images processed and saved at: /kaggle/working/21F1005989_images


In [11]:
# 8) CREATE SUBMISSION (via submission.py)
# -----------------------------
import os
import numpy as np
import pandas as pd
from PIL import Image

def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L') 
            image_array = np.array(image).flatten()[::8]
            # Replace 'test_' with 'gt_' in the ID
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')




In [12]:
folder_path = '/kaggle/working/21F1005989_images'
output_csv = 'submission.csv'
images_to_csv(folder_path, output_csv)

Successfully saved to submission.csv
